In [95]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

from tagging.sugest import sugerir_rotulos
from extract import extract_data
import io, os


In [96]:
contas = ["Conta Corrente XP", "Cartão de Crédito XP"]

In [97]:
def get_upload_widget() -> widgets.FileUpload:
    return widgets.FileUpload(
        accept=".csv",
        multiple=False,
        description="📁 Selecionar CSV",
        style={"font_weight": "bold"},
        layout=widgets.Layout(padding="8px 15px", width="auto"),
    )


def criar_campos_input():
    conta = widgets.Dropdown(options=contas, layout=widgets.Layout(width='30%'))
    tipo = widgets.Text(description="Tipo:", placeholder="Digite o tipo aqui", layout=widgets.Layout(width='30%'))
    categoria = widgets.Text(description="Categoria:", placeholder="Digite a categoria aqui", layout=widgets.Layout(width='30%'))
    subcategoria = widgets.Text(description="Subcategoria:", placeholder="Digite a subcategoria aqui", layout=widgets.Layout(width='30%'))
    nome = widgets.Text(description="Nome:", placeholder="Digite o nome aqui", layout=widgets.Layout(width='100%'))
    notas = widgets.Textarea(description="Notas:", placeholder="Digite notas aqui", layout=widgets.Layout(width='100%', height='100px'))
    return conta, tipo, categoria, subcategoria, nome, notas

def mostrar_item(row, index, size):
    clear_output(wait=True)
    texto = f"### Item {index+1} / {size}\n"
    for k, v in row.items():
        texto += f"**{k}**: {v}  \n"
    display(Markdown(texto))


In [ ]:
def save_row(row, labels):
    if not os.path.exists("entry.csv"):
        with open("entry.csv", "w", encoding="utf-8-sig") as f:
            f.write(','.join(labels) + '\n')

    with open("entry.csv", "a", encoding="utf-8-sig") as f:
        values = [str(row.get(label, '')) for label in labels]
        f.write(','.join(values) + '\n')

In [ ]:
def handle_upload(_):
    if value := upload_widget.value:
        file_info = next(iter(value.values()))
        content = file_info["content"]
        df = extract_data(io.BytesIO(content))
        process_row(df, 0, [])


def exibir_upload_widget():
    global upload_widget
    upload_widget = get_upload_widget()
    upload_widget.observe(handle_upload, names="value")
    display(upload_widget)


def process_row(df, index, labels):
    row = df.iloc[index]
    mostrar_item(row, index, len(df))

    conta, tipo, categoria, subcategoria, nome, notas = criar_campos_input()

    if rotulos := sugerir_rotulos(row["Data"], row["Valor"], row["Destino / Origem"], row["Descricao"]):
        tipo.value = rotulos.get("Tipo", "")
        categoria.value = rotulos.get("Categoria", "")
        subcategoria.value = rotulos.get("Subcategoria", "")
        nome.value = rotulos.get("Nome", "")

    linha_superior = widgets.HBox([conta, tipo, categoria, subcategoria], layout=widgets.Layout(gap='10px'))
    linha_media = widgets.HBox([nome])
    linha_inferior = widgets.HBox([notas], layout=widgets.Layout(width='100%'))

    formulario = widgets.VBox([linha_superior, linha_media, linha_inferior], layout=widgets.Layout(width='100%', gap='10px'))

    button = widgets.Button(description="Confirmar")
    output = widgets.Output()

    def on_confirm(_):

        tipo_value = tipo.value.strip()
        categoria_value = categoria.value.strip()
        subcategoria_value= subcategoria.value.strip()
        nome_value = nome.value.strip()

        if not tipo_value:
            with output:
                clear_output()
                print("Por favor, insira um tipo antes de confirmar.")
            return

        if not categoria_value:
            with output:
                clear_output()
                print("Por favor, insira uma categoria antes de confirmar.")
            return
        
        if not subcategoria_value:
            with output:
                clear_output()
                print("Por favor, insira uma subcategoria antes de confirmar.")
            return
        
        if not nome_value:
            with output:
                clear_output()
                print("Por favor, insira um nome antes de confirmar.")
            return
        
        labels = ["Data","Valor","Destino / Origem","Descricao","Tipo","Categoria","Subcategoria","Nome","Conta","Notas"]
        row_data = {
            "Data": row["Data"],
            "Valor": row["Valor"],
            "Destino / Origem": row["Destino / Origem"],
            "Descricao": row["Descricao"],
            "Tipo": tipo_value,
            "Categoria": categoria_value,
            "Subcategoria": subcategoria_value,
            "Nome": f'"{nome_value}"',
            "Conta": contas.index(conta.value)+1 if conta.value in contas else conta.value,
            "Notas": notas.value.strip()
        }
        save_row(row_data, labels)
        if index + 1 < len(df):
            process_row(df, index + 1, labels)
        else:
            clear_output()
            exibir_upload_widget()

    button.on_click(on_confirm)
    display(formulario, button, output)


In [ ]:
upload_widget = get_upload_widget()
upload_widget.observe(handle_upload, names="value")
display(upload_widget)

### Item 4 / 11
**Data**: 2024-01-31 09:25:27  
**Valor**: -900.0  
**Destino / Origem**: para conta investimento  
**Descricao**: Transferência enviada para conta investimento  
**Tipo**: Transferência  


Button(description='Confirmar', style=ButtonStyle())

Output()